In [ ]:
import sys
import os
import gc
import copy
# Thêm đường dẫn đến thư mục MPAX-fix
mpax_path = os.path.abspath("mpax")  # Đảm bảo đường dẫn đúng với vị trí thư mục
sys.path.append(mpax_path)



In [ ]:
from mpax import *
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
import timeit
from pathlib import Path
import gurobipy as gp
import jax
import jax.numpy as jnp
import pytest
from jax import config, jit, vmap
from jax.experimental.sparse import BCOO
from jax.sharding import PartitionSpec as P
from pulp import LpProblem
from pulp2mat import convert_all

from mpax.mp_io import create_lp, create_qp_from_gurobi
from mpax.r2hpdhg import r2HPDHG
from mpax.nes2_pdhg import nes2_pdhg


In [ ]:
pytest_cachepath= os.path.abspath(".pytest_cache")

In [ ]:
import ipytest
ipytest.autoconfig()


| Model       | Describe                                           |
|---------------|----------------------------------------------------|
| `r2HPDHG`     | Use original weight that paper talk about      |
| `r2HPDHG_weight`  | Update weight                |

_______________________________________________________________________________________________________
# Thử với tập LP cỡ lớn

In [ ]:
import gurobipy as gp
import os
os.environ["XLA_FLAGS"] = "--xla_gpu_deterministic_ops=true"
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.5" 
import jax.numpy as jnp
from jax.sharding import PartitionSpec as P
from jax.lib import xla_bridge
from mpax.mp_io import create_lp, create_qp_from_gurobi
from mpax.nes_pdhg import nes_pdhg
from mpax.nes2_pdhg import nes2_pdhg
from mpax.nes3_pdhg import nes3_pdhg
from mpax.r2hpdhg import r2HPDHG
from mpax.rapdhg import raPDHG
from mpax.rhpdhg import rHPDHG
from mpax.pdhg import PDHG
from mpax.r2hpdhg_mpax import r2HPDHG_mpax
input_dir = "/home/kobv/lhiep/test-vscode/MPAX_main/medium_instances"
# Initialize CSV log
import time
import gurobipy as gp
import jax.numpy as jnp
# Thư mục chứa các file .mps đã presolve
import os
os.environ["JAX_LOG_COMPILES"] = "1"
jax.config.update("jax_enable_x64", True)


In [ ]:
import os
import time
import json
import gc
import gurobipy as gp
from mpax.mp_io import create_qp_from_gurobi

log = []
count = 0

def log_gpu_mem(label):
    try:
        import subprocess
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits"],
            stdout=subprocess.PIPE,
            text=True
        )
        print(f"[{label}] GPU Mem Used: {result.stdout.strip()} MB")
    except:
        pass

for fname in os.listdir(input_dir):
    if not (fname.endswith(".mps") or fname.endswith(".lp")):
        continue

    count += 1
    # if count < 41:
    #     print(f"✅ Bỏ qua bài {count}: {fname}")
    #     continue
    # if count in [44]:
    #     log.append((fname, "undefined", "timeout"))
    #     continue
    # if count > 80:
    #     print("✅ Đã xử lý đủ 40 bài")
    #     break
    if count != 17:
        continue

    print(f"🔄 Bài {count}: {fname}")
    path = os.path.join(input_dir, fname)

    try:
        # ===== Warm-up để kích hoạt JIT (nếu có) =====
        gurobi_model = gp.read(path)
        gurobi_model.setParam("Seed", 36)
        gurobi_model.setParam("Threads", 1)
        gurobi_model.setParam("OutputFlag", 0)

        gurobi_model = gurobi_model.relax()
        lp = create_qp_from_gurobi(gurobi_model)


        solver_warmup = nes3_pdhg(iteration_limit=100,eps_abs=1e-6, eps_rel=1e-6)
        _ = solver_warmup.optimize(lp)

        print(f"🚀 Warm-up xong {fname}")
        # ===== Chính thức solve =====
        log_gpu_mem("🔍 Trước solve")

        solver = nes3_pdhg(iteration_limit=2500000, eps_abs=1e-6, eps_rel=1e-6,verbose= True,l2_norm_rescaling=True)
        start = time.time()
        result = solver.optimize(lp)
        elapsed = time.time() - start
        
        status = int(result.termination_status)
        obj_val = float(getattr(result, "primal_objective", float("nan")))

        if status == 0:
            print(f"✅ {fname} | Optimal | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 1:
            print(f"✅ {fname} | Feasible (suboptimal) | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 2:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 3:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        else:
            print(f"❌ {fname} | No solution | Time: timeout")
            log.append((fname, "undefined", "timeout"))
        del solver_warmup,solver
        gc.collect()
        time.sleep(1)

    except Exception as e:
        print(f"❌ Lỗi xử lý {fname}: {e}")
        log.append((fname, None, None))

    log_gpu_mem("📦 Sau solve")
    time.sleep(1)
    gc.collect()

# ===== Ghi log ra file =====
with open("solve_log_nes_pdhg_3_e6_med.txt", "a") as f:
    for fname, obj, t in log:
        f.write(f"{fname}\tObj: {obj}\tTime: {t}\n")

In [ ]:
import os
import time
import json
import gc
import gurobipy as gp
from mpax.mp_io import create_qp_from_gurobi

log = []
count = 0

def log_gpu_mem(label):
    try:
        import subprocess
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits"],
            stdout=subprocess.PIPE,
            text=True
        )
        print(f"[{label}] GPU Mem Used: {result.stdout.strip()} MB")
    except:
        pass

for fname in os.listdir(input_dir):
    if not (fname.endswith(".mps") or fname.endswith(".lp")):
        continue

    count += 1
    # if count < 41:
    #     print(f"✅ Bỏ qua bài {count}: {fname}")
    #     continue
    # if count in [44]:
    #     log.append((fname, "undefined", "timeout"))
    #     continue
    # if count > 80:
    #     print("✅ Đã xử lý đủ 40 bài")
    #     break
    if count != 17:
        continue

    print(f"🔄 Bài {count}: {fname}")
    path = os.path.join(input_dir, fname)

    try:
        # ===== Warm-up để kích hoạt JIT (nếu có) =====
        gurobi_model = gp.read(path)
        gurobi_model.setParam("Seed", 36)
        gurobi_model.setParam("Threads", 1)
        gurobi_model.setParam("OutputFlag", 0)

        gurobi_model = gurobi_model.relax()
        lp = create_qp_from_gurobi(gurobi_model)


        solver_warmup = nes_pdhg(iteration_limit=100,eps_abs=1e-6, eps_rel=1e-6)
        _ = solver_warmup.optimize(lp)

        print(f"🚀 Warm-up xong {fname}")
        # ===== Chính thức solve =====
        log_gpu_mem("🔍 Trước solve")

        solver = nes_pdhg(iteration_limit=2500000, eps_abs=1e-6, eps_rel=1e-6,verbose= True,l2_norm_rescaling=True)
        start = time.time()
        result = solver.optimize(lp)
        elapsed = time.time() - start
        
        status = int(result.termination_status)
        obj_val = float(getattr(result, "primal_objective", float("nan")))

        if status == 0:
            print(f"✅ {fname} | Optimal | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 1:
            print(f"✅ {fname} | Feasible (suboptimal) | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 2:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 3:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        else:
            print(f"❌ {fname} | No solution | Time: timeout")
            log.append((fname, "undefined", "timeout"))
        del solver_warmup,solver
        gc.collect()
        time.sleep(1)

    except Exception as e:
        print(f"❌ Lỗi xử lý {fname}: {e}")
        log.append((fname, None, None))

    log_gpu_mem("📦 Sau solve")
    time.sleep(1)
    gc.collect()

# ===== Ghi log ra file =====
with open("solve_log_nes_pdhg_3_e6_med.txt", "a") as f:
    for fname, obj, t in log:
        f.write(f"{fname}\tObj: {obj}\tTime: {t}\n")

In [ ]:
import os
import time
import json
import gc
import gurobipy as gp
from mpax.mp_io import create_qp_from_gurobi

log = []
count = 0

def log_gpu_mem(label):
    try:
        import subprocess
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits"],
            stdout=subprocess.PIPE,
            text=True
        )
        print(f"[{label}] GPU Mem Used: {result.stdout.strip()} MB")
    except:
        pass

for fname in os.listdir(input_dir):
    if not (fname.endswith(".mps") or fname.endswith(".lp")):
        continue

    count += 1
    # if count < 41:
    #     print(f"✅ Bỏ qua bài {count}: {fname}")
    #     continue
    # if count in [44]:
    #     log.append((fname, "undefined", "timeout"))
    #     continue
    # if count > 80:
    #     print("✅ Đã xử lý đủ 40 bài")
    #     break
    if count != 9:
        continue

    print(f"🔄 Bài {count}: {fname}")
    path = os.path.join(input_dir, fname)

    try:
        # ===== Warm-up để kích hoạt JIT (nếu có) =====
        gurobi_model = gp.read(path)
        gurobi_model.setParam("Seed", 36)
        gurobi_model.setParam("Threads", 1)
        gurobi_model.setParam("OutputFlag", 0)

        gurobi_model = gurobi_model.relax()
        lp = create_qp_from_gurobi(gurobi_model)


        solver_warmup = nes2_pdhg(iteration_limit=100,eps_abs=1e-6, eps_rel=1e-6)
        _ = solver_warmup.optimize(lp)

        print(f"🚀 Warm-up xong {fname}")
        # ===== Chính thức solve =====
        log_gpu_mem("🔍 Trước solve")

        solver = nes2_pdhg(iteration_limit=2500000, eps_abs=1e-6, eps_rel=1e-6,verbose= True,l2_norm_rescaling=True)
        start = time.time()
        result = solver.optimize(lp)
        elapsed = time.time() - start
        
        status = int(result.termination_status)
        obj_val = float(getattr(result, "primal_objective", float("nan")))

        if status == 0:
            print(f"✅ {fname} | Optimal | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 1:
            print(f"✅ {fname} | Feasible (suboptimal) | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 2:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 3:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        else:
            print(f"❌ {fname} | No solution | Time: timeout")
            log.append((fname, "undefined", "timeout"))
        del solver_warmup,solver
        gc.collect()
        time.sleep(1)

    except Exception as e:
        print(f"❌ Lỗi xử lý {fname}: {e}")
        log.append((fname, None, None))

    log_gpu_mem("📦 Sau solve")
    time.sleep(1)
    gc.collect()

# ===== Ghi log ra file =====
with open("solve_log_nes_pdhg_3_e6_med.txt", "a") as f:
    for fname, obj, t in log:
        f.write(f"{fname}\tObj: {obj}\tTime: {t}\n")

In [ ]:
import os
import time
import json
import gc
import gurobipy as gp
from mpax.mp_io import create_qp_from_gurobi

log = []
count = 0

def log_gpu_mem(label):
    try:
        import subprocess
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits"],
            stdout=subprocess.PIPE,
            text=True
        )
        print(f"[{label}] GPU Mem Used: {result.stdout.strip()} MB")
    except:
        pass

for fname in os.listdir(input_dir):
    if not (fname.endswith(".mps") or fname.endswith(".lp")):
        continue

    count += 1
    if count < 41:
        print(f"✅ Bỏ qua bài {count}: {fname}")
        continue
    if count in [44]:
        log.append((fname, "undefined", "timeout"))
        continue
    if count > 80:
        print("✅ Đã xử lý đủ 40 bài")
        break

    print(f"🔄 Bài {count}: {fname}")
    path = os.path.join(input_dir, fname)

    try:
        # ===== Warm-up để kích hoạt JIT (nếu có) =====
        gurobi_model = gp.read(path)
        gurobi_model.setParam("Seed", 36)
        gurobi_model.setParam("Threads", 1)
        gurobi_model.setParam("OutputFlag", 0)

        gurobi_model = gurobi_model.relax()
        lp = create_qp_from_gurobi(gurobi_model)


        solver_warmup = nes_pdhg(iteration_limit=100,eps_abs=1e-6, eps_rel=1e-6)
        _ = solver_warmup.optimize(lp)

        print(f"🚀 Warm-up xong {fname}")
        # ===== Chính thức solve =====
        log_gpu_mem("🔍 Trước solve")

        solver = nes_pdhg(iteration_limit=2500000, eps_abs=1e-6, eps_rel=1e-6,verbose= True,l2_norm_rescaling=True)
        start = time.time()
        result = solver.optimize(lp)
        elapsed = time.time() - start
        
        status = int(result.termination_status)
        obj_val = float(getattr(result, "primal_objective", float("nan")))

        if status == 0:
            print(f"✅ {fname} | Optimal | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 1:
            print(f"✅ {fname} | Feasible (suboptimal) | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 2:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 3:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        else:
            print(f"❌ {fname} | No solution | Time: timeout")
            log.append((fname, "undefined", "timeout"))
        del solver_warmup,solver
        gc.collect()
        time.sleep(1)

    except Exception as e:
        print(f"❌ Lỗi xử lý {fname}: {e}")
        log.append((fname, None, None))

    log_gpu_mem("📦 Sau solve")
    time.sleep(1)
    gc.collect()

# ===== Ghi log ra file =====
with open("solve_log_nes_pdhg_2_e6_med.txt", "a") as f:
    for fname, obj, t in log:
        f.write(f"{fname}\tObj: {obj}\tTime: {t}\n")

In [ ]:
import os
import time
import json
import gc
import gurobipy as gp
from mpax.mp_io import create_qp_from_gurobi

log = []
count = 0

def log_gpu_mem(label):
    try:
        import subprocess
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits"],
            stdout=subprocess.PIPE,
            text=True
        )
        print(f"[{label}] GPU Mem Used: {result.stdout.strip()} MB")
    except:
        pass

for fname in os.listdir(input_dir):
    if not (fname.endswith(".mps") or fname.endswith(".lp")):
        continue

    count += 1
    if count < 41:
        print(f"✅ Bỏ qua bài {count}: {fname}")
        continue
    if count in [44]:
        log.append((fname, "undefined", "timeout"))
        continue
    if count > 80:
        print("✅ Đã xử lý đủ 40 bài")
        break

    print(f"🔄 Bài {count}: {fname}")
    path = os.path.join(input_dir, fname)

    try:
        # ===== Warm-up để kích hoạt JIT (nếu có) =====
        gurobi_model = gp.read(path)
        gurobi_model.setParam("Seed", 36)
        gurobi_model.setParam("Threads", 1)
        gurobi_model.setParam("OutputFlag", 0)

        gurobi_model = gurobi_model.relax()
        lp = create_qp_from_gurobi(gurobi_model)


        solver_warmup = r2HPDHG(iteration_limit=100,eps_abs=1e-6, eps_rel=1e-6)
        _ = solver_warmup.optimize(lp)

        print(f"🚀 Warm-up xong {fname}")
        # ===== Chính thức solve =====
        log_gpu_mem("🔍 Trước solve")

        solver = r2HPDHG(iteration_limit=2500000, eps_abs=1e-6, eps_rel=1e-6,verbose= True,l2_norm_rescaling=True)
        start = time.time()
        result = solver.optimize(lp)
        elapsed = time.time() - start
        
        status = int(result.termination_status)
        obj_val = float(getattr(result, "primal_objective", float("nan")))

        if status == 0:
            print(f"✅ {fname} | Optimal | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 1:
            print(f"✅ {fname} | Feasible (suboptimal) | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 2:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 3:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        else:
            print(f"❌ {fname} | No solution | Time: timeout")
            log.append((fname, "undefined", "timeout"))
        del solver_warmup,solver
        gc.collect()
        time.sleep(1)

    except Exception as e:
        print(f"❌ Lỗi xử lý {fname}: {e}")
        log.append((fname, None, None))

    log_gpu_mem("📦 Sau solve")
    time.sleep(1)
    gc.collect()

# ===== Ghi log ra file =====
with open("solve_log_r2hpdhg_e6_med.txt", "a") as f:
    for fname, obj, t in log:
        f.write(f"{fname}\tObj: {obj}\tTime: {t}\n")

In [ ]:
import os
import time
import json
import gc
import gurobipy as gp
from mpax.mp_io import create_qp_from_gurobi

log = []
count = 0

def log_gpu_mem(label):
    try:
        import subprocess
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits"],
            stdout=subprocess.PIPE,
            text=True
        )
        print(f"[{label}] GPU Mem Used: {result.stdout.strip()} MB")
    except:
        pass

for fname in os.listdir(input_dir):
    if not (fname.endswith(".mps") or fname.endswith(".lp")):
        continue
    
    count += 1
    if count < 41:
        print(f"✅ Bỏ qua bài {count}: {fname}")
        continue
    if count in [44]:
        log.append((fname, "undefined", "timeout"))
        continue
    if count > 80:
        print("✅ Đã xử lý đủ 40 bài")
        break

    print(f"🔄 Bài {count}: {fname}")
    path = os.path.join(input_dir, fname)

    try:
        # ===== Warm-up để kích hoạt JIT (nếu có) =====
        gurobi_model = gp.read(path)
        gurobi_model.setParam("Seed", 36)
        gurobi_model.setParam("Threads", 1)
        gurobi_model.setParam("OutputFlag", 0)

        gurobi_model = gurobi_model.relax()
        lp = create_qp_from_gurobi(gurobi_model)


        solver_warmup = r2HPDHG_mpax(iteration_limit=100,eps_abs=1e-6, eps_rel=1e-6)
        _ = solver_warmup.optimize(lp)

        print(f"🚀 Warm-up xong {fname}")
        # ===== Chính thức solve =====
        log_gpu_mem("🔍 Trước solve")

        solver = r2HPDHG_mpax(iteration_limit=2500000, eps_abs=1e-6, eps_rel=1e-6,verbose= True,l2_norm_rescaling=True)
        start = time.time()
        result = solver.optimize(lp)
        elapsed = time.time() - start
        
        status = int(result.termination_status)
        obj_val = float(getattr(result, "primal_objective", float("nan")))

        if status == 0:
            print(f"✅ {fname} | Optimal | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 1:
            print(f"✅ {fname} | Feasible (suboptimal) | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 2:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 3:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        else:
            print(f"❌ {fname} | No solution | Time: timeout")
            log.append((fname, "undefined", "timeout"))
        del solver_warmup,solver
        gc.collect()
        time.sleep(1)

    except Exception as e:
        print(f"❌ Lỗi xử lý {fname}: {e}")
        log.append((fname, None, None))

    log_gpu_mem("📦 Sau solve")
    time.sleep(1)
    gc.collect()

# ===== Ghi log ra file =====
with open("solve_log_r2hpdhg_mpax_e6_med.txt", "a") as f:
    for fname, obj, t in log:
        f.write(f"{fname}\tObj: {obj}\tTime: {t}\n")

In [ ]:
import os
import time
import json
import gc
import gurobipy as gp
from mpax.mp_io import create_qp_from_gurobi

log = []
count = 0

def log_gpu_mem(label):
    try:
        import subprocess
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits"],
            stdout=subprocess.PIPE,
            text=True
        )
        print(f"[{label}] GPU Mem Used: {result.stdout.strip()} MB")
    except:
        pass

for fname in os.listdir(input_dir):
    if not (fname.endswith(".mps") or fname.endswith(".lp")):
        continue

    count += 1
    if count < 41:
        print(f"✅ Bỏ qua bài {count}: {fname}")
        continue
    if count in [44]:
        log.append((fname, "undefined", "timeout"))
        continue
    if count > 80:
        print("✅ Đã xử lý đủ 40 bài")
        break
    print(f"🔄 Bài {count}: {fname}")
    path = os.path.join(input_dir, fname)

    try:
        # ===== Warm-up để kích hoạt JIT (nếu có) =====
        gurobi_model = gp.read(path)
        gurobi_model.setParam("Seed", 36)
        gurobi_model.setParam("Threads", 1)
        gurobi_model.setParam("OutputFlag", 0)

        gurobi_model = gurobi_model.relax()
        lp = create_qp_from_gurobi(gurobi_model)


        solver_warmup = rHPDHG(iteration_limit=100,eps_abs=1e-6, eps_rel=1e-6)
        _ = solver_warmup.optimize(lp)

        print(f"🚀 Warm-up xong {fname}")
        # ===== Chính thức solve =====
        log_gpu_mem("🔍 Trước solve")

        solver = rHPDHG(iteration_limit=2500000, eps_abs=1e-6, eps_rel=1e-6,verbose= True,l2_norm_rescaling=True)
        start = time.time()
        result = solver.optimize(lp)
        elapsed = time.time() - start
        
        status = int(result.termination_status)
        obj_val = float(getattr(result, "primal_objective", float("nan")))

        if status == 0:
            print(f"✅ {fname} | Optimal | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 1:
            print(f"✅ {fname} | Feasible (suboptimal) | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 2:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        elif status == 3:
            print(f"✅ {fname} | Obj: {obj_val:.6f} | Time: {elapsed:.2f}s")
            log.append((fname, obj_val, elapsed))
        else:
            print(f"❌ {fname} | No solution | Time: timeout")
            log.append((fname, "undefined", "timeout"))
        del solver_warmup,solver
        gc.collect()
        time.sleep(1)

    except Exception as e:
        print(f"❌ Lỗi xử lý {fname}: {e}")
        log.append((fname, None, None))

    log_gpu_mem("📦 Sau solve")
    time.sleep(1)
    gc.collect()

# ===== Ghi log ra file =====
with open("solve_log_rhpdhg_e6_med.txt", "a") as f:
    for fname, obj, t in log:
        f.write(f"{fname}\tObj: {obj}\tTime: {t}\n")